#  [問題1] 

- 業務や身の回りの適当な問題を線形計画問題や整数計画問題(一部の変数のみに整数条件がつく問題でもよい), 2次計画問題やネットワーク最適化問題に定式化してみよ．
- もし，可能であれば，解いてみよ

## データ収集
- 日経225、S&P500、BTCに関する過去５年分の終値データ（1,223件）をYahoo! Financeから取得し、ドル/円の為替レートを用いて日本円に換算する。

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Yahoo Financeからデータを取得
symbols = ["^N225", "^GSPC", "BTC-USD"]
period = "5y"  # 5年分

# 各ティッカーのデータを取得
df_close = pd.DataFrame()
for symbol in symbols:
    ticker = yf.Ticker(symbol)
    data = ticker.history(period=period)  
    data.index = data.index.date
    df_close[symbol] = data["Close"]

# 欠損値を前日のデータで補完
df_close = df_close.ffill()

# 為替レート（USD/JPY）の取得
usd_jpy_ticker = yf.Ticker("JPY=X")
usd_jpy = usd_jpy_ticker.history(period=period)["Close"]
usd_jpy.index = usd_jpy.index.tz_localize(None)
usd_jpy = usd_jpy.reindex(df_close.index, method="ffill")

# USDをJPYに変換
df_close["^GSPC/JPY"] = df_close["^GSPC"] * usd_jpy
df_close["BTC/JPY"] = df_close["BTC-USD"] * usd_jpy
df_close = df_close.drop(columns=["^GSPC", "BTC-USD"])

# 結果を確認
print(df_close.head())
print(df_close.tail())
print(len(df_close))

                   ^N225      ^GSPC/JPY       BTC/JPY
2020-01-31  23205.179688            NaN           NaN
2020-02-03  22971.939453  352091.955127  1.007158e+06
2020-02-04  23084.589844  358157.853016  9.971627e+05
2020-02-05  23319.560547  364971.805336  1.052160e+06
2020-02-06  23873.589844  367413.498144  1.068468e+06
                   ^N225      ^GSPC/JPY       BTC/JPY
2025-01-27  39565.800781  936376.522675  1.589955e+07
2025-01-28  39016.871094  939941.348816  1.569731e+07
2025-01-29  39414.781250  939469.068054  1.613197e+07
2025-01-30  39513.968750  942063.425661  1.625178e+07
2025-01-31  39572.488281  930827.519041  1.578031e+07
1223


In [288]:
# csvにダンプ
df_close.to_csv("df_close.csv", encoding="utf-8-sig", index=True)

In [281]:
# 欠損値を確認
df_close.isnull().sum()

^N225        0
^GSPC/JPY    1
BTC/JPY      1
dtype: int64

## リターンとリスクの評価

In [295]:
# 日次リターンを計算
df_return = df_close.pct_change().dropna()

# 各資産の平均リターン
expected_returns = df_return.mean()

# リターンの共分散行列
cov_matrix = df_return.cov()

# 各資産のリスク（標準偏差）を計算
std_devs = np.sqrt(np.diag(cov_matrix))

print("日次リターンの平均:\n", expected_returns, "\n")
print("共分散行列:\n", cov_matrix, "\n")

print("各資産の標準偏差:\n")
print(pd.Series(std_devs, index=df_return.columns))

日次リターンの平均:
 ^N225        0.000540
^GSPC/JPY    0.000905
BTC/JPY      0.003156
dtype: float64 

共分散行列:
               ^N225  ^GSPC/JPY   BTC/JPY
^N225      0.000188   0.000059  0.000069
^GSPC/JPY  0.000059   0.000216  0.000246
BTC/JPY    0.000069   0.000246  0.001761 

各資産の標準偏差:

^N225        0.013695
^GSPC/JPY    0.014699
BTC/JPY      0.041967
dtype: float64


## シャープレシオの最大化

In [286]:
# シャープレシオを最大化する最適化関数
def negative_sharpe_ratio(weights, expected_returns, cov_matrix):
    portfolio_return = np.sum(weights * expected_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return -(portfolio_return / portfolio_volatility)

# 最適化の実行
num_assets = len(expected_returns)
initial_guess = np.ones(num_assets) / num_assets  # 初期ウェイト（均等分割）
bounds = [(0, 1) for _ in range(num_assets)]  # 各資産のウェイト範囲
constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}  # 合計ウェイトが1になる制約

result = minimize(negative_sharpe_ratio, initial_guess, args=(expected_returns, cov_matrix), 
                  method='SLSQP', bounds=bounds, constraints=constraints)

# 最適なウェイトを取得
optimal_weights = result.x

# 投資額（3万円）の分配
investment = 30000
amount_invested = optimal_weights * investment

print("最適なポートフォリオのウェイト:", np.round(optimal_weights, 4))
print("各資産への投資額:", np.round(amount_invested, 2))

最適なポートフォリオのウェイト: [0.32 0.4  0.28]
各資産への投資額: [ 9731.13 12004.22  8264.65]


# ５つの暗号資産に分配する場合（参考）

In [241]:
import yfinance as yf
import pandas as pd

# 取得する暗号資産のティッカー（USD建て）
symbols = ["BTC-USD", "ETH-USD", "LTC-USD", "XRP-USD", "ADA-USD"]
period = "max"  # 1年分

# 終値を保存するためのデータフレームを作成
df_close = pd.DataFrame()

for symbol in symbols:
    ticker = yf.Ticker(symbol)  # 暗号資産の情報を取得
    df_close[symbol] = ticker.history(period=period)["Close"]  # 過去1年分の終値を取得し、データフレームに格納
    
# USD/JPYの為替レートを取得
usd_jpy_ticker = yf.Ticker("JPY=X")
usd_jpy = usd_jpy_ticker.history(period=period)["Close"]  # 過去1年分の為替レートを取得
usd_jpy = usd_jpy.reindex(df_close.index, method="ffill")  # 為替データの取得分を前日値で補完

# USDをJPYに変換
df_close_jpy = df_close.multiply(usd_jpy, axis=0)  # df_closeにusd_jpyを行単位で計算

df_close_jpy.columns = [symbol.replace("-USD", "/JPY") for symbol in df_close_jpy.columns]

# 時間部分を削除し、日付だけ表示
df_close_jpy.index = df_close_jpy.index.date

print(df_close_jpy.head())
print(df_close_jpy.tail())
print(len(df_close_jpy))

                 BTC/JPY  ETH/JPY     LTC/JPY  XRP/JPY  ADA/JPY
2014-09-17  49003.340394      NaN  542.023627      NaN      NaN
2014-09-18  46058.530813      NaN  508.422389      NaN      NaN
2014-09-19  42953.804916      NaN  470.861415      NaN      NaN
2014-09-20  44488.755545      NaN  466.364674      NaN      NaN
2014-09-21  43391.727584      NaN  461.956129      NaN      NaN
                 BTC/JPY        ETH/JPY       LTC/JPY     XRP/JPY     ADA/JPY
2025-01-29  1.613197e+07  484255.032947  17996.673974  477.300771  146.400781
2025-01-30  1.625178e+07  503958.021199  20158.390743  485.847962  149.120536
2025-01-31  1.578031e+07  508252.763176  19726.405244  467.749107  145.194658
2025-02-01  1.551077e+07  480525.084701  18324.461370  444.488025  138.370473
2025-02-02  1.530476e+07  478533.147723  17764.296108  440.018764  136.538527
3792


In [242]:
# 日次リターンを計算
df_return = df_close_jpy.pct_change().dropna()

# リターンの最初の5行を表示
print(df_return.head())

             BTC/JPY   ETH/JPY   LTC/JPY   XRP/JPY   ADA/JPY
2017-11-10 -0.078043 -0.071929 -0.082414 -0.055201 -0.158032
2017-11-11 -0.039368  0.051555  0.051353  0.019115  0.011726
2017-11-12 -0.064101 -0.021523 -0.052933 -0.062211 -0.126107
2017-11-13  0.105446  0.031428  0.043378  0.033754  0.079317
2017-11-14  0.010727  0.065090  0.019992  0.030459  0.015449


In [243]:
# リターンの共分散行列を計算
cov_matrix = df_return.cov()

# 共分散行列を表示
print(cov_matrix)

          BTC/JPY   ETH/JPY   LTC/JPY   XRP/JPY   ADA/JPY
BTC/JPY  0.001341  0.001309  0.001346  0.001133  0.001370
ETH/JPY  0.001309  0.002103  0.001843  0.001656  0.001857
LTC/JPY  0.001346  0.001843  0.002643  0.001829  0.001950
XRP/JPY  0.001133  0.001656  0.001829  0.003797  0.002408
ADA/JPY  0.001370  0.001857  0.001950  0.002408  0.004431


In [244]:
import numpy as np

# 各資産の平均リターンを計算
expected_returns = df_return.mean()

# 資産数
num_assets = len(symbols)

# ランダムなウェイトを生成（合計が1になるように調整）
weights = np.random.random(num_assets)
weights /= np.sum(weights)

# ポートフォリオの期待リターンを計算
portfolio_return = np.sum(weights * expected_returns)

# ポートフォリオのリスク（標準偏差）を計算
portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

print("ポートフォリオの期待リターン:", portfolio_return)
print("ポートフォリオのリスク:", portfolio_volatility)

ポートフォリオの期待リターン: 0.002273741440516255
ポートフォリオのリスク: 0.043590290007335164


In [245]:
import numpy as np
from scipy.optimize import minimize

# 無リスク利子率（例：1%）
risk_free_rate = 0.01

# シャープレシオを最大化する最適化関数
def negative_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate):
    # ポートフォリオの期待リターン
    portfolio_return = np.sum(weights * expected_returns)
    # ポートフォリオのリスク（標準偏差）
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    # シャープレシオの計算（無リスク利子率を考慮）
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
    # シャープレシオを最大化するために、最小化問題にするため負の値を返す
    return -sharpe_ratio

# 最適化
num_assets = len(symbols)
initial_guess = np.ones(num_assets) / num_assets  # 初期のウェイト（等分割）
bounds = [(0, 1) for _ in range(num_assets)]  # 各資産のウェイトは0から1の間
constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}  # ウェイトの合計が1になる制約

# 最適化を実行
result = minimize(negative_sharpe_ratio, initial_guess, args=(expected_returns, cov_matrix, risk_free_rate),
                  method='SLSQP', bounds=bounds, constraints=constraints)

# 最適なウェイトを取得
optimal_weights = result.x

# 投資額（3万円）の割り当て
investment = 30000
amount_invested = optimal_weights * investment

# 結果表示
print("最適なポートフォリオのウェイト:", np.round(optimal_weights, 4))
print("各暗号資産への投資額:", np.round(amount_invested, 2))

最適なポートフォリオのウェイト: [0. 0. 0. 0. 1.]
各暗号資産への投資額: [    0.     0.     0.     0. 30000.]


In [246]:
import numpy as np

# 数値表示をわかりやすくする（少数第2位まで表示）
np.set_printoptions(precision=2, suppress=True)

# 結果表示
print("\n最適なポートフォリオのウェイト:")
for symbol, weight in zip(symbols, optimal_weights):  # zip()symbol, weightの組み合わせをタプルで取り出す関数
    print(f"{symbol}: {weight*100:.2f}%")

print("\n各暗号資産への投資額:")
for symbol, amount in zip(symbols, amount_invested):
    print(f"{symbol}: ¥{amount:,.0f}")


最適なポートフォリオのウェイト:
BTC-USD: 0.00%
ETH-USD: 0.00%
LTC-USD: 0.00%
XRP-USD: 0.00%
ADA-USD: 100.00%

各暗号資産への投資額:
BTC-USD: ¥0
ETH-USD: ¥0
LTC-USD: ¥0
XRP-USD: ¥0
ADA-USD: ¥30,000
